# README
このファイルはgoogle colab上で使用できます．上から順にセルを実行していくと問題なく動くと思います．  
  
【やったこと】  
- 課題1(adrianから自分(cody)までたどり着けるか？)に関して，BFSとDFSの実装による実行時間比較  
- 課題1(発展ver1)として，自分(cody)から何人の人にたどり着けるか，何人の人から自分(cody)にたどり着けるか調査
- アップデートの可能性あり．

## 必要な準備

In [0]:
from google.colab import auth, drive, files, output
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("/content/drive/My Drive/STEP2020")

## SNS解析 - データの準備


In [0]:
#nicknamesの読み込み
f1 = open('txt/sns/nicknames.txt')
nicknames = f1.read()
f1.close()
nicknames = nicknames.split("\n")
#何故かreとsplit('\\')がうまくいかなかったな..
for i in range(len(nicknames)):
    if i < 10:
        nicknames[i] = nicknames[i][2:]
    else:
        nicknames[i] = nicknames[i][3:]
#nicknames #確認用

In [0]:
#名前から番号を検索できるようにしておく．
from collections import defaultdict
name2num = defaultdict(int)
for i in range(len(nicknames)):
    name2num[nicknames[i]] = i
#name2num #確認用

In [0]:
#linksの読み込み
f1 = open('txt/sns/links.txt')
links = f1.read()
f1.close()
links = links.split("\n")
links = tuple([tuple(map(int, links[i].split('\t'))) for i in range(len(links))])
#links #確認用

In [0]:
#隣接リスト作成
direct_neighbors = [[] for i in range(len(nicknames))]
for link in links:
    direct_neighbors[link[0]].append(link[1])
#direct_neighbors #確認用

## 課題1 - adrianからcodyまでたどり着けるか？
結論を先にまとめると，実行時間は今回はBFSの方が少し(約40倍，4/1000秒)速かった．おそらくグラフの形による？と思われる．  
BFS: 0.00016641616821289062  
DFS: 0.004436969757080078

### BFS編

In [0]:
import time
from collections import deque

def bfs(direct_neighbors, visited, start, goal):
    q = deque()
    q.extend(direct_neighbors[start])
    visited[start] = True
    while len(q) > 0:
        node_num = q.popleft()
        visited[node_num] = True
        for next_node in direct_neighbors[node_num]:
            if next_node == goal:
                return True
            elif (visited[next_node] == False) and (next_node not in q):
                q.append(next_node)
    return False

start = name2num['adrian']
goal = name2num['cody']
visited = [False for i in range(len(nicknames))]

bfs_time_start = time.time()
if bfs(direct_neighbors, visited, start, goal):
    print('adrianからcodyまでたどり着ける')
else:
    print('adrianからcodyまでたどり着けない')
bfs_time_finish = time.time()

print('実行時間：', bfs_time_finish - bfs_time_start)

adrianからcodyまでたどり着ける
実行時間： 0.00016641616821289062


### DFS編

In [0]:
import time
from collections import deque

def dfs(direct_neighbors, visited, start, goal):
    for next_node in direct_neighbors[start]:
        if next_node == goal:
            return True
        elif visited[next_node] == False:
            visited[next_node] = True
            if dfs(direct_neighbors, visited, next_node, goal):
                return True
    return False

start = name2num['adrian']
goal = name2num['cody']
visited = [False for i in range(len(nicknames))]

dfs_time_start = time.time()
if dfs(direct_neighbors, visited, start, goal):
    print('adrianからcodyまでたどり着ける')
else:
    print('adrianからcodyまでたどり着けない')
dfs_time_finish = time.time()

print('実行時間：', dfs_time_finish - dfs_time_start)

adrianからcodyまでたどり着ける
実行時間： 0.004436969757080078


## 課題1(発展ver1) - codyと繋がっているのは？
### codyから探索を始める編
実装のポイント：探索の終了判定はqueueの中のnodeがなくなった時として，このノードに到達したら終了のようにしないようにする．それにより，for文でgoalを指定して探索するより計算量が減らせる．


In [0]:
import time
from collections import deque

def bfs(direct_neighbors, visited, start):
    q = deque()
    q.extend(direct_neighbors[start])
    visited[start] = True
    while len(q) > 0:
        node_num = q.popleft()
        visited[node_num] = True
        for next_node in direct_neighbors[node_num]:
            if (visited[next_node] == False) and (next_node not in q):
                q.append(next_node)

start = name2num['cody']
visited = [False for i in range(len(nicknames))]
not_yet = []

bfs_time_start = time.time()
bfs(direct_neighbors, visited, start)
print('codyから到達できる人数：', sum(visited))
for i in range(len(visited)):
    if visited[i] == False:
        not_yet.append(nicknames[i])
print('codyからたどれない人：', not_yet)
bfs_time_finish = time.time()

print('実行時間：', bfs_time_finish - bfs_time_start)

codyから到達できる人数： 51
codyからたどれない人： ['betty', 'carolyn', 'lawrence']
実行時間： 0.00044536590576171875


### codyまで探索してみる編
実装のポイント：startはfor文で回すのだが，一度探索したところともう一度探索すると効率が悪いので，reachedにぶつかったら探索を終了するようにして計算量を減らした．ただ，頑張ればもう少し計算量を減らせそう(まだダブルサーチを減らせそう)ということに気づいたので，実装がうまくいったら更新する可能性あり．

In [0]:
import time
from collections import deque

def bfs(direct_neighbors, visited, reached, start, goal):
    q = deque()
    q.extend(direct_neighbors[start])
    visited[start] = True
    while len(q) > 0:
        node_num = q.popleft()
        visited[node_num] = True
        for next_node in direct_neighbors[node_num]:
            if (next_node == goal) or (reached[next_node] == True):
                return True
            elif (visited[next_node] == False) and (next_node not in q):
                q.append(next_node)
    return False

goal = name2num['cody']
reached = [False for i in range(len(nicknames))]
cannot_reach = []

bfs_time_start = time.time()
for i in range(len(reached)):
    visited = [False for i in range(len(nicknames))]
    if bfs(direct_neighbors, visited, reached, i, goal):
        reached[i] = True
print('codyまで到達できる人数：', sum(reached))
for i in range(len(reached)):
    if reached[i] == False:
        cannot_reach.append(nicknames[i])
print('codyからたどれない人：', cannot_reach)
bfs_time_finish = time.time()

print('実行時間：', bfs_time_finish - bfs_time_start)

codyまで到達できる人数： 51
codyからたどれない人： ['betty', 'carolyn', 'lawrence']
実行時間： 0.0005311965942382812
